In [75]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.dates import MonthLocator, DateFormatter
import re
from sqlalchemy import create_engine, text
import warnings
warnings.filterwarnings("ignore")  # Suppress all warnings

#Reemplazar end point
pd.options.display.max_rows = 20
DB_USERNAME = 'developers'
DB_PASSWORD = '12345678'
DB_HOST = ''
DB_PORT= '5432'
DB_NAME = 'cnc'
engine=create_engine(f'postgresql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}', max_overflow=20, client_encoding='utf8')

def runQuery(sql):
    result = engine.connect().execution_options(isolation_level="AUTOCOMMIT").execute((text(sql)))
    return pd.DataFrame(result.fetchall(), columns=result.keys())

In [76]:
query1="""
select edbp.codenc, edbp.competencias_funcionales, edbp.numeros,
edbp.lectura_voz_alta, edbp.fluidez_lectura, edbp.cartografia, edbp.competencias_organizacionales, 
edbp.sumar, edbp.coherencia_entre_numeros, 
edbs.como_conocio_cnc, edbs.fecha_registro, 
edbs.ciudad as "ciudad_encuestador", edbs.experiencia, edbs.tipo,
edbs.barrio, edbs.fecha_nacimiento, edbs.localidad,
edbs.educacion_formal, edbs.estado_educacion, edbs.nombre_educacion
from ecar_dwh.ecar_dwh_base_psicotecnica edbp 
full outer join ecar_dwh.ecar_dwh_base_sociodemografica edbs on edbs.codenc=edbp.codenc"""

query2="""
SELECT * FROM efectivas_final;
"""

query3="""
select edbma.* from ecar_dwh.ecar_dwh_base_marcador_automatico edbma """

query4="""
select *  from ecar_dwh.ecar_dwh_base_muestra_ECAR_mensual_cuotas """

query5="""
select *  from ecar_dwh.ecar_dwh_base_fechas """


#df: Códigos left sociodemografica y psicotecnica
df=runQuery(query1)
#df2: Codigos left efectivas 
df2=runQuery(query2)
#df3: ´Marcador automático left códigos
df3=runQuery(query3)
#df4: Cuotas
df4=runQuery(query4)
#df5: fechas
df5=runQuery(query5)

### We solve issues with sociodemoprahic and psicological datasets,  it had diferent values for some pollsters, so we took the max value 

In [77]:
df1=df.groupby(['codenc', 'fecha_registro']).agg({'competencias_funcionales':"mean", 'numeros':"mean", 'lectura_voz_alta':"mean",
       'fluidez_lectura':"mean", 'cartografia':"mean", 'competencias_organizacionales':"mean",
       'sumar':"mean", 'coherencia_entre_numeros':"mean", "como_conocio_cnc":"max", 
       'fecha_nacimiento':"max", 'localidad':"max", 'educacion_formal':"max", 'estado_educacion':"max",
       'nombre_educacion':"max"}).reset_index()
df1=df1[~((df1['codenc']=='Encuestador 135')&(df1['fecha_registro']=='2018-04-01'))]
df1

,codenc,fecha_registro,competencias_funcionales,numeros,lectura_voz_alta,fluidez_lectura,cartografia,competencias_organizacionales,sumar,coherencia_entre_numeros,como_conocio_cnc,fecha_nacimiento,localidad,educacion_formal,estado_educacion,nombre_educacion
0,Encuestador 1,2017-05-25,3.73,4.17,4.0,4.0,3.75,4.0,5.0,2.50,internet - computrabajo.com,1979-11-05,nan,profesional,aplazado,filología inglés
1,Encuestador 10,2016-07-13,4.13,5.00,4.0,4.0,3.75,4.0,5.0,5.00,voz a voz- referido por un conocido,NaN,nan,nan,nan,nan
2,Encuestador 100,2016-07-14,3.61,1.94,5.0,5.0,3.75,3.0,0.0,3.33,internet - zonajobs.com,NaN,nan,nan,nan,nan
3,Encuestador 102,2017-11-14,1.80,4.17,0.0,0.0,3.75,4.0,5.0,2.50,internet - computrabajo.com,1998-12-03,rafael uribe uribe,tecnólogo,nan,tecnología en control ambiental
4,Encuestador 104,2019-05-30,1.25,4.58,0.0,0.0,1.25,5.0,5.0,5.00,voz a voz- referido por un conocido,1999-09-12,antonio nariño,bachiller,culminado,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,Encuestador 91,2017-09-29,4.26,4.58,4.0,4.0,1.25,4.0,5.0,3.75,voz a voz- referido por un conocido,NaN,soacha,bachiller,culminado,nan
182,Encuestador 93,2017-10-30,1.80,3.33,0.0,0.0,3.75,4.0,5.0,0.00,otros,1999-07-12,bosa,bachiller,culminado,nan
183,Encuestador 95,2017-02-19,4.25,5.00,4.0,4.0,2.50,4.0,5.0,5.00,otros,1994-08-20,puente aranda,bachiller,culminado,nan
184,Encuestador 96,2016-07-14,4.19,5.00,4.0,4.0,3.75,4.0,5.0,5.00,voz a voz- referido por un conocido,NaN,nan,nan,nan,nan


In [78]:
cols=[['ciudad bolivar', 'usme', 'tunjuelito'], ['bosa', 'kennedy', 'puente aranda', 'fontibon'], 
['santa fé', 'san cristóbal', 'la victoria', 'antonio nariño','rafael uribe uribe', 'mártires', 'la candelaria'],
['chapinero', 'engativa', 'suba', 'teusaquillo', 'usaquen'],  ['comuna 3', 'soacha']]
new_cols=['sur', 'sur_occidente', 'centro oriente','norte','otros']

for i,j in zip(cols, new_cols):
    df1['localidad'].replace(i,j, inplace=True)

In [102]:
import datetime
import numpy as np
today = datetime.date.today()
df1['age']=df1['fecha_nacimiento'].apply(lambda x: str((pd.Timestamp(today)-pd.Timestamp(x))/365).split(' ')[0])
df1['age'].replace('NaT', np.nan, inplace=True)
df1

,codenc,fecha_registro,competencias_funcionales,numeros,lectura_voz_alta,fluidez_lectura,cartografia,competencias_organizacionales,sumar,coherencia_entre_numeros,como_conocio_cnc,fecha_nacimiento,localidad,educacion_formal,estado_educacion,nombre_educacion,age
0,Encuestador 1,2017-05-25,3.73,4.17,4.0,4.0,3.75,4.0,5.0,2.50,internet - computrabajo.com,1979-11-05,nan,profesional,aplazado,filología inglés,40
1,Encuestador 10,2016-07-13,4.13,5.00,4.0,4.0,3.75,4.0,5.0,5.00,voz a voz- referido por un conocido,NaN,nan,nan,nan,nan,NaN
2,Encuestador 100,2016-07-14,3.61,1.94,5.0,5.0,3.75,3.0,0.0,3.33,internet - zonajobs.com,NaN,nan,nan,nan,nan,NaN
3,Encuestador 102,2017-11-14,1.80,4.17,0.0,0.0,3.75,4.0,5.0,2.50,internet - computrabajo.com,1998-12-03,centro oriente,tecnólogo,nan,tecnología en control ambiental,21
4,Encuestador 104,2019-05-30,1.25,4.58,0.0,0.0,1.25,5.0,5.0,5.00,voz a voz- referido por un conocido,1999-09-12,centro oriente,bachiller,culminado,nan,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,Encuestador 91,2017-09-29,4.26,4.58,4.0,4.0,1.25,4.0,5.0,3.75,voz a voz- referido por un conocido,NaN,otros,bachiller,culminado,nan,NaN
182,Encuestador 93,2017-10-30,1.80,3.33,0.0,0.0,3.75,4.0,5.0,0.00,otros,1999-07-12,sur_occidente,bachiller,culminado,nan,21
183,Encuestador 95,2017-02-19,4.25,5.00,4.0,4.0,2.50,4.0,5.0,5.00,otros,1994-08-20,sur_occidente,bachiller,culminado,nan,25
184,Encuestador 96,2016-07-14,4.19,5.00,4.0,4.0,3.75,4.0,5.0,5.00,voz a voz- referido por un conocido,NaN,nan,nan,nan,nan,NaN


In [130]:
df1.to_csv('../data_etls/ecar_dwh_base_sociodemografica_y_psicotecnica.csv',index=False)

# We solve issues with data "automatic caller" because it had diferent values in the time columns by some pollsters, so we took the total value 

In [5]:
df3['archivo']=df3['archivo'].astype(str)
df3['mes']=df3['archivo'].str.split('_',0).str[1]
df3['año']=df3['archivo'].str.split('_',0).str[2].str.split('.',0).str[0]
df3.drop('archivo', axis=1, inplace=True)

VCD = df3.drop_duplicates()
VCD['n_mes'] = VCD['mes']

List1=['espera','hablado','disponible','pausas','muerto','cliente','conectado']
for i in List1:
    VCD[i]=(VCD[i]).astype(float)

VCD['n_mes'] = VCD['n_mes'].replace({'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4, 'Mayo': 5, 'Junio': 6, 'Julio': 7, 'Agosto': 8, 
                     'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12})
VCD['dia']=1
lista1=list(range(1,13))
lista2=[31,28,31,30,31,30,31,31,30,31,30,31]
for i,j in zip(lista1,lista2):
    VCD.loc[VCD['n_mes']==i,['dia']]=j


VCD['fecha'] = VCD['año'].astype(str)+ ' ' +VCD['n_mes'].astype(str)+ ' ' +VCD['dia'].astype(str)
VCD['fecha'] = pd.to_datetime(VCD['fecha']) 


VCD_id = VCD.groupby(['codenc','fecha']).agg({'calls': ['count', 'sum'], 'espera': ['sum'], 'hablado': ['sum'], 'disponible': ['sum'], 
                                          'pausas': ['sum'], 'muerto': ['sum'], 'cliente': ['sum'],'conectado':['sum']}).reset_index() #, 'conectado': ['sum']
VCD_id.dtypes
VCD_id.columns=['codenc', 'fecha', 'N', 'calls', 'espera', 'hablado', 'disponible', 'pausas', 'muerto', 'cliente','conectado']

#Recalulando los porcentajes
VCD_id['total'] = VCD_id['espera'] + VCD_id['hablado'] + VCD_id['disponible'] + VCD_id['pausas']
VCD_id['porcentaje_espera'] = (VCD_id[('espera')] / VCD_id['total'] )*100
VCD_id['porcentaje_hablado'] = (VCD_id['hablado'] / VCD_id['total'] )*100
VCD_id['porcentaje_disponible'] = (VCD_id['disponible'] / VCD_id['total'])*100 
VCD_id['porcentaje_muerto'] = (VCD_id['muerto'] / VCD_id['total'] )*100
VCD_id['porcentaje_pausas'] = (VCD_id['pausas'] / VCD_id['total'] )*100
VCD_id=VCD_id[['codenc', 'calls','total', 'espera', 'porcentaje_espera', 'hablado',
       'porcentaje_hablado', 'disponible', 'porcentaje_disponible', 'pausas',
       'porcentaje_pausas', 'muerto', 'porcentaje_muerto', 'cliente',
       'conectado', 'fecha']]

In [132]:

for i in ['total', 'espera', 'hablado', 'disponible', 'pausas', 'muerto', 'cliente', 'conectado']:
    VCD_id[i]=VCD_id[i].astype(int)
VCD_id.to_csv('../data_etls/ecar_dwh_base_marcador_final.csv',index=False)

In [107]:
df2['mes']= pd.DatetimeIndex(df2['fecha']).month
df2['dia']= pd.DatetimeIndex(df2['fecha']).day
df2['año']= pd.DatetimeIndex(df2['fecha']).year
df_dias_trabajados=df2.groupby(['codenc', 'año', 'mes']).agg({'dia':'nunique', 
                                         'duracion':'mean', 'Genero':"count"}).reset_index().rename(columns={'dia':'dias_trabajados', 'Genero':'efectivas_mes'})

df_dias_trabajados['dia']=1
lista1=list(range(1,13))
lista2=[31,28,31,30,31,30,31,31,30,31,30,31]
for i,j in zip(lista1,lista2):
    df_dias_trabajados.loc[df_dias_trabajados['mes']==i,['dia']]=j
df_dias_trabajados['fecha']= df_dias_trabajados['año'].astype(str)+ ' ' +df_dias_trabajados['mes'].astype(str)+ ' ' +df_dias_trabajados['dia'].astype(str)
df_dias_trabajados['fecha']=pd.to_datetime(df_dias_trabajados['fecha']) 
df_dias_trabajados['efectividad_ajustada']=(df_dias_trabajados['efectivas_mes']/df_dias_trabajados['dias_trabajados'])*24
df_dias_trabajados[df_dias_trabajados['codenc']=='Encuestador 270']

,codenc,año,mes,dias_trabajados,duracion,efectivas_mes,dia,fecha,efectividad_ajustada
828,Encuestador 270,2019,2,21,863.461039,154,28,2019-02-28,176.000000
829,Encuestador 270,2019,3,23,748.351598,219,31,2019-03-31,228.521739
830,Encuestador 270,2019,4,24,676.802239,268,30,2019-04-30,268.000000
831,Encuestador 270,2019,5,23,676.016807,238,31,2019-05-31,248.347826
832,Encuestador 270,2019,6,22,770.298755,241,30,2019-06-30,262.909091
833,Encuestador 270,2019,7,25,630.602273,176,31,2019-07-31,168.960000
834,Encuestador 270,2019,8,24,649.221591,176,31,2019-08-31,176.000000
835,Encuestador 270,2019,9,25,576.737089,213,30,2019-09-30,204.480000
836,Encuestador 270,2019,10,24,674.279167,240,31,2019-10-31,240.000000
837,Encuestador 270,2019,11,24,648.620321,187,30,2019-11-30,187.000000


In [ ]:
df2['fecha']=pd.to_datetime(df2['fecha'])
def count_var(df2, var):
    data=df2[['fecha', 'codenc']].drop_duplicates()
    for i in df2[var].unique(): 
        dfs=df2[df2[var]==i].groupby([pd.Grouper(key='fecha',freq='M'),'codenc']).agg({'telelink':'count'}).reset_index().rename(columns={'telelink':'efectivas '+str(i)})
        data=dfs.merge(data, on=['fecha', 'codenc'], how='outer')
    return data

df2_genero=count_var(df2, 'genero')
df2_estrato=count_var(df2, 'estrato')
df2_ciudad=count_var(df2, 'ciudad')
df2_edad=count_var(df2, 'cod_rango_edad')

In [ ]:
dt=df_dias_trabajados.merge(df2_genero,on=['codenc','fecha'],how='left')
dt=dt.merge(df2_estrato, on=['codenc','fecha'],how='left')
dt=dt.merge(df2_ciudad, on=['codenc','fecha'],how='left')
dt=dt.merge(df2_edad,  on=['codenc','fecha'],how='left')
dt=dt.fillna(0)

var=dt.columns.tolist()[9:]

for i in var:
    dt[str(i)]=(dt[i]/dt['dias_trabajados'])*24
    

In [ ]:
VCD_efectivas=VCD_id.merge(df_dias_trabajado,on=['codenc','fecha'],how='left')
VCD_efectivas['calls']=(VCD_efectivas['calls']/VCD_efectivas['dias_trabajados'])*24
VCD_efectivas


In [ ]:
meses_trabajados=VCD_efectivas.groupby(['codenc']).agg({'fecha':"count"}).reset_index().rename(columns={'fecha':'meses_trabajados'})
meses_trabajados

In [ ]:
VCD_efectivas.merge(meses_trabajados, on='codenc', how='left')

In [ ]:
cols=['calls', 'total', 'espera', 'hablado', 'disponible', 'pausas', 'muerto', 'duracion', 
      'dias_trabajados','efectividad_ajustada', 'efectivas Mujer', 'efectivas Hombre', 'efectivas  Alto',
       'efectivas  Medio Alto', 'efectivas  Medio Medio', 'efectivas  Bajo',
       'efectivas  Medio Bajo', 'efectivas Tunja', 'efectivas Popayán',
       'efectivas Montería', 'efectivas Manizales', 'efectivas Armenia',
       'efectivas Villavicencio', 'efectivas Pasto', 'efectivas Cúcuta',
       'efectivas Santa Marta', 'efectivas Ibagué', 'efectivas Neiva',
       'efectivas Pereira', 'efectivas Cartagena', 'efectivas Cali',
       'efectivas Bucaramanga', 'efectivas Barranquilla', 'efectivas Medellín',
       'efectivas Bogotá', 'efectivas 5', 'efectivas 2', 'efectivas 4',
       'efectivas 3', 'efectivas 1']
marcador_efectivas_total=VCD_efectivas.groupby('codenc')[cols].mean().reset_index().rename(columns={'0': 'mean '+ str(i)})
marcador_efectivas_total=marcador_efectivas_total.merge(meses_trabajados, on='codenc', how='left')
df_total=marcador_efectivas_total.merge(df1, on='codenc', how='left')


In [ ]:
df_total['como_conocio_cnc'].replace({'no recuerda':'otros',
                'internet - tuempleo.com': 'internet - computrabajo.com',
                'internet - pagina en facebook del cnc':'internet - pagina web del cnc',
                'fue contactado por el cnc':'otros', 
                'internet - facebook- grupos de trabajo':'otros'}, inplace=True)

In [ ]:
cols=[['ciudad bolivar', 'usme', 'tunjuelito'], ['bosa', 'kennedy', 'puente aranda', 'fontibon'], 
['santa fé', 'san cristóbal', 'la victoria', 'antonio nariño','rafael uribe uribe', 'mártires', 'la candelaria'],
['chapinero', 'engativa', 'suba', 'teusaquillo', 'usaquen'],  ['comuna 3', 'soacha', 'no_disponible']]
new_cols=['sur', 'sur_occidente', 'centro oriente','norte','otros']

for i,j in zip(cols, new_cols):
    df_total['localidad'].replace(i,j, inplace=True)

In [ ]:
cols=[['tecnico auxiliar contable y financiero','administracion de empresas', 'asistencia administrativa', 
     'comercio y negocios internacionales','gestion logistica','gestión de empresas agropecuarias', 
     'tecnico en asistencia y organizacion de archivos', 'tecnología de gestión integrada', 
     'tecnólogo en gestión administrativa', 'secretariado administrativo y de gerencia', 
     'secretariado ejecutivo', 'tecnología en gestión empresarial'],  
      ['ingenieria ambiental', 'manejo agroforestal', 'tecnologo en gestión ambiental y servicio publicos',
       'licenciatura en biología', 'técnico en procesamiento de frutas y hortalizas', 'administración de servicios de salud',
       'habitos de vida saludable', 'psicología', 'trabajo social', 'tecnología en procesos administrativos de salud', 
       'ingenieria en procesos quimicos', 'tecnología en control ambiental','tecnico en seguridad ocupacional']]
cols_new=['ciencias administrativas y contables', 'ciencias ambientales y salud']
for i, j in zip(cols, cols_new):
    df_total['nombre_educacion'].replace(i,j, inplace=True)

In [ ]:
cols2=list(df_total['nombre_educacion'][~(df_total['nombre_educacion'].isin(['ciencias administrativas y contables', 'ciencias ambientales y salud', 'no_disponible']))])
df_total['nombre_educacion'].replace(cols2, 'otros', inplace=True)

In [ ]:
df_total=df_total.drop(['experiencia', 'ciudad_encuestador', 'tipo', 'barrio', 'fecha_nacimiento', 'fecha_registro'], axis=1)

In [ ]:
df_total.columns=['codenc','calls', 'total', 'espera', 'hablado', 'disponible', 'pausas', 'muerto', 'duracion_efectivas', 
        'dias_trabajados', 'efectividad_ajustada', 'efectivas_Mujer', 'efectivas_Hombre', 'efectivas_Alto',
       'efectivas_Medio_Alto', 'efectivas_Medio_Medio', 'efectivas_Bajo',
       'efectivas_Medio_Bajo', 'efectivas_Tunja', 'efectivas_Popayán',
       'efectivas_Montería', 'efectivas_Manizales', 'efectivas_Armenia',
       'efectivas_Villavicencio','efectivas_Pasto', 'efectivas_Cúcuta',
       'efectivas_Santa_Marta', 'efectivas_Ibagué', 'efectivas_Neiva',
       'efectivas_Pereira', 'efectivas_Cartagena', 'efectivas_Cali',
       'efectivas_Bucaramanga','efectivas_Barranquilla', 'efectivas_Medellín',
       'efectivas_Bogotá', 'efectivas_rango_edad_5', 'efectivas_rango_edad_2',
      'efectivas_rango_edad_4', 'efectivas_rango_edad_3', 'efectivas_rango_edad_1',
        'meses_trabajados','competencias_funcionales', 'numeros', 
      'lectura_voz_alta','fluidez_lectura', 'cartografia', 'competencias_organizacionales',
       'sumar', 'coherencia_entre_numeros', 'como_conocio_cnc','localidad', 
        'educacion_formal', 'estado_educacion','nombre_educacion']
      
df_total.to_csv('../data_etls/ecar_dwh_base_modelo.csv', index=False)